In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
import plotly.graph_objects as graph_objects
import findspark

findspark.init()

# spark_context = SparkContext.getOrCreate()
spark_context = SparkContext('local', 'R1_A2')
spark_session = SparkSession(spark_context)
sqlContext = SQLContext(spark_context)

/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
text_file = sqlContext.read.format('com.databricks.spark.csv').\
    options(header='true', inferschema='true', quote='"', delimiter=',').\
    load('./work/data/TraficoAereoGt.csv')

rddfiltro = text_file.rdd.map(tuple)
rddGen = rddfiltro.map(lambda word: (word[2], word[3]))
# rddGen.take(10)

In [3]:
rddConteo = rddGen.reduceByKey(lambda a, b: a+b)
print(f'Conteo total -> {rddConteo.collect()}')

Conteo total -> [('Coatepeque', 167223), ('Huehuetenango', 198734), ('Poptun', 173559), ('Puerto Barros', 203204), ('Mundo Maya', 170385), ('La Aurora', 150513), ('Puerto San Jose', 162211), ('Quetzaltenango', 194403), ('Retalhuleu', 174374), ('San Marcos', 208806)]


In [4]:
rddNombres = rddConteo.map(lambda x: (x[0]))
print(rddNombres.collect())

rddTotales = rddConteo.map(lambda x: (x[1]))
print(rddTotales.collect())

['Coatepeque', 'Huehuetenango', 'Poptun', 'Puerto Barros', 'Mundo Maya', 'La Aurora', 'Puerto San Jose', 'Quetzaltenango', 'Retalhuleu', 'San Marcos']
[167223, 198734, 173559, 203204, 170385, 150513, 162211, 194403, 174374, 208806]


In [5]:
graph = graph_objects.Figure(
    data=graph_objects.Pie(
        labels=rddNombres.collect(),
        values=rddTotales.collect()
    ))

graph.update_layout(
    title_text='Aterrizajes por Aeropuerto',
    title_font_size=30)

graph.update_traces(
    hoverinfo='label+percent',
    textinfo='value',
    textfont_size=20)

graph.write_html('./work/reports/R1_A2.html', auto_open=True)